In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import transformers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaModel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import sys
import joblib
import matplotlib.pyplot as plt

sys.path.append('/content/drive/MyDrive/skripsi')
!ls /content/drive/MyDrive/skripsi


 check_lang.py					   kmeans_en
 cluster_labels					   model
 cluster_labels_en				   pca
 cluster_sinta2.ipynb				   pca_en
 cluster_sinta.ipynb				   predict_multibert_kmeans_finetuned.ipynb
 cluster_sinta_v1.ipynb				   predict_RoBERTa_kmeans_finetuned.ipynb
'Copy of cluster_sinta3.ipynb'			   Preprocessing_data_sinta.ipynb
'Copy of predict_RoBERTa_kmeans_finetuned.ipynb'   Preprocessing.py
'Copy of Preprocessing_data_sinta.ipynb'	   __pycache__
'Copy of Untitled0.ipynb'			  'referensi skripsi'
 data						  'Salinan cluster_sinta_v1.ipynb'
 data_jid					   silhouette_scores
 embeddings					   silhouette_scores_cluster
 embeddings_2d					   silhouette_scores_en
 embeddings_2d_en				   Skripsi
 embeddings_en					   src
 finetuned_multibert2.ipynb			   tokenizer
 finetuned_pipeline.py				   tokenizer_en
 finetuning_RoBERTa.ipynb			   Untitled0.ipynb
 full_en					   Untitled1.ipynb
 functions.py					   Untitled2.ipynb
 GUI.py						   Untitled3.ipynb
 Hasil						  'versi revisian'
 kmean

In [ ]:
from Preprocessing import preprocess_text

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/skripsi/data/combined_data.csv')
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df

,jid,desc,title,date,journal,data_cleaned,eissn,jid_umum
0,1,Belakangan ini Lembaga PAUD berbondong-bondong...,Kemampuan Bahasa Inggris Awal pada Periode Lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Early English language skills in the linguisti...,25498959,1
1,1,The assessment process carried out in schools ...,Strengthening Early Childhood Learning Outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,strengthening early childhood learning outcome...,25498959,1
2,1,Kewajiban guru PAUD untuk memberikan layanan b...,Eksplorasi Deskriptif tentang Layanan Bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,descriptive exploration of guidance and counse...,25498959,1
3,1,Pandemi Covid-19 merambah sejak awal tahun 202...,Perspektif Orang Tua terhadap Pelaksanaan Les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Parents' perspective on the implementation of ...,25498959,1
4,1,Dalam upaya mencapai Sustainable Development G...,Strategi Pencegahan dan Penanganan Stunting Mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,multidimensional stunting prevention and handl...,25498959,1
...,...,...,...,...,...,...,...,...
8663,12,The purpose of this study to examine the effec...,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",2017-05-15,E-JURNAL AKUNTANSI,the effect of financial performance of general...,23028556,12
8664,12,The level of understanding of accounting to be...,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",2017-08-06,E-JURNAL AKUNTANSI,the influence of emotional intelligence intell...,23028556,12
8665,12,The purpose of this study was to examine the i...,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",2017-02-15,E-JURNAL AKUNTANSI,"the influence of company size, the complexity ...",23028556,12
8666,12,The purpose of this study is to determine the ...,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,2016-11-10,E-JURNAL AKUNTANSI,performance of the Bali tourism office based o...,23028556,12


In [ ]:
df[df.data_cleaned.isna()]

,jid,desc,title,date,journal,data_cleaned,eissn,jid_umum


In [ ]:
title_unik = list(df.journal.unique())
title_unik

['JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI',
 'JURNAL CENDEKIA : JURNAL PENDIDIKAN MATEMATIKA',
 'INTERNATIONAL JOURNAL OF ELEMENTARY EDUCATION',
 'JURNAL BISNIS DAN AKUNTANSI',
 'JURNAL AKUNTANSI DAN KEUANGAN',
 'JURNAL PENDIDIKAN TEKNIK MESIN UNDIKSHA',
 'INTERNATIONAL JOURNAL OF BASIC AND APPLIED SCIENCE',
 'JURNAL KESEHATAN MASYARAKAT',
 'GADJAH MADA INTERNATIONAL JOURNAL OF BUSINESS',
 'JURNAL KESEHATAN ANDALAS',
 'E-JOURNAL OF CULTURAL STUDIES',
 'E-JURNAL AKUNTANSI']

In [ ]:
df.data_cleaned[0]

"Early English language skills in the linguistic period of early childhood lately early childhood education institutions have flocked to provide English language learning for children in the linguistic period this study aims to determine children's early ability to listen and pronounce English vocabulary in the linguistic period this study is a descriptive study using a quantitative approach the population of this study includes children in kindergarten then the researcher conducted a sampling with a saturated sampling technique with the number of children this study was conducted in September March in collaboration with class teachers the data collection activities of this study were carried out through observation with a participatory model the description of the results of this study is divided into two namely the ability to listen to early English vocabulary namely with the category starting to develop then the ability to pronounce early English vocabulary is in the category starti

In [ ]:
df['jid_umum'] = df['jid']
df['jid_umum'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [ ]:
journal_id = {title_unik[i] : i + 1 for i in range(len(title_unik))}
df['jid'] = df.journal.apply(lambda x : journal_id[x])
df

,jid,desc,title,date,journal,data_cleaned,eissn,jid_umum
0,1,Belakangan ini Lembaga PAUD berbondong-bondong...,Kemampuan Bahasa Inggris Awal pada Periode Lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Early English language skills in the linguisti...,25498959,1
1,1,The assessment process carried out in schools ...,Strengthening Early Childhood Learning Outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,strengthening early childhood learning outcome...,25498959,1
2,1,Kewajiban guru PAUD untuk memberikan layanan b...,Eksplorasi Deskriptif tentang Layanan Bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,descriptive exploration of guidance and counse...,25498959,1
3,1,Pandemi Covid-19 merambah sejak awal tahun 202...,Perspektif Orang Tua terhadap Pelaksanaan Les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Parents' perspective on the implementation of ...,25498959,1
4,1,Dalam upaya mencapai Sustainable Development G...,Strategi Pencegahan dan Penanganan Stunting Mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,multidimensional stunting prevention and handl...,25498959,1
...,...,...,...,...,...,...,...,...
8663,12,The purpose of this study to examine the effec...,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",2017-05-15,E-JURNAL AKUNTANSI,the effect of financial performance of general...,23028556,12
8664,12,The level of understanding of accounting to be...,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",2017-08-06,E-JURNAL AKUNTANSI,the influence of emotional intelligence intell...,23028556,12
8665,12,The purpose of this study was to examine the i...,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",2017-02-15,E-JURNAL AKUNTANSI,"the influence of company size, the complexity ...",23028556,12
8666,12,The purpose of this study is to determine the ...,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,2016-11-10,E-JURNAL AKUNTANSI,performance of the Bali tourism office based o...,23028556,12


In [ ]:
df.to_csv('/content/drive/MyDrive/skripsi/data/data_sinta_cleaned3.csv', index=False)

In [ ]:
'''
1 'JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI',
2 'JURNAL CENDEKIA : JURNAL PENDIDIKAN MATEMATIKA',
3 'INTERNATIONAL JOURNAL OF ELEMENTARY EDUCATION',
4 'JURNAL BISNIS DAN AKUNTANSI',
5 'JURNAL AKUNTANSI DAN KEUANGAN',
6 'JURNAL PENDIDIKAN TEKNIK MESIN UNDIKSHA',
7 'INTERNATIONAL JOURNAL OF BASIC AND APPLIED SCIENCE',
8 'JURNAL KESEHATAN MASYARAKAT',
9 'GADJAH MADA INTERNATIONAL JOURNAL OF BUSINESS',
10 'JURNAL KESEHATAN ANDALAS',
11 'E-JOURNAL OF CULTURAL STUDIES',
12 'E-JURNAL AKUNTANSI'
 '''

jurnal_id = 12
data_jid = df[df['jid']==jurnal_id]
data = data_jid['data_cleaned']
data

,data_cleaned
7521,profitability and corporate social responsibil...
7522,information technology is a system that helps ...
7523,the influence of managerial ownership and inst...
7524,the effect of productive asset quality car lev...
7525,the effect of professionalism hardiness person...
...,...
8663,the effect of financial performance of general...
8664,the influence of emotional intelligence intell...
8665,"the influence of company size, the complexity ..."
8666,performance of the Bali tourism office based o...


In [ ]:
# Count the number of articles for the specified journal ID.
num_articles = len(df[df['jid'] == jurnal_id])

print(f"The number of articles in journal ID {jurnal_id} is: {num_articles}")

The number of articles in journal ID 12 is: 1147


In [ ]:
class RobertaClassifier(nn.Module):
    def __init__(self, num_labels):
        super(RobertaClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs['last_hidden_state']
        # x = self.classifier(x)
        return x

In [ ]:
# Load pre-trained RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaClassifier(12)
model.load_state_dict(torch.load('/content/drive/MyDrive/skripsi/model/finetuned_model_roberta_en.pt',map_location=torch.device('cpu')))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like roberta-base is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
# Simpan tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/skripsi/tokenizer_en')

In [ ]:
def tokenize_data(texts, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_length,
                            pad_to_max_length=True,
                            return_attention_mask=True,
                            return_tensors='pt'
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

# Tokenize data
input_ids, attention_masks = tokenize_data(data, tokenizer)

In [ ]:
# Create dataloader
class ArticleDataset(Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx]
        }

    def __len__(self):
        return len(self.input_ids)

dataset = ArticleDataset(input_ids, attention_masks)

In [ ]:
# Create dataloader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
model.to(device)
# Set model to evaluation mode
model.eval()

In [ ]:
# Embedding
embeddings = []

with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        embeddings.append(outputs.cpu().numpy())

embeddings = np.concatenate(embeddings, axis=0)

embeddings_dir = '/content/drive/MyDrive/skripsi/embeddings_en' # Tentukan direktori penyimpanan
os.makedirs(embeddings_dir, exist_ok=True) # Buat direktori jika belum ada
np.save(os.path.join(embeddings_dir, f'embeddings_{jurnal_id}.npy'), embeddings) # Simpan embeddings
print(f"Embeddings saved to {embeddings_dir}")

In [ ]:
embeddings.shape

In [ ]:
embeddings[0][0]

In [ ]:
# Mengubah array embeddings menjadi matriks dua dimensi
X = embeddings.reshape(embeddings.shape[0], -1)

pca = PCA(n_components=2, random_state=0)
X = pca.fit_transform(X)

In [ ]:
embeddings_2d_dir = '/content/drive/MyDrive/skripsi/embeddings_2d_en'
os.makedirs(embeddings_2d_dir, exist_ok=True)

np.save(os.path.join(embeddings_2d_dir, f'embeddings_2d_{jurnal_id}.npy'), X)

print(f"Embeddings 2D saved to {embeddings_2d_dir}")

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_scores = []
for k in range(2, 7):  # Mencoba nilai k dari 2 hingga 6
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For n_clusters = {k}, the average silhouette_score is : {silhouette_avg}")

best_k = np.argmax(silhouette_scores) + 2  # Menambahkan 2 karena perulangan dimulai dari 2
lowest_k = np.argmin(silhouette_scores) + 2
print(f"The best value of k is : {best_k} with silhouette score {silhouette_scores[best_k-2]}")
print(f"The lowest value of k is : {lowest_k} with silhouette score {silhouette_scores[lowest_k-2]}")

In [ ]:
import matplotlib.pyplot as plt
# Visualisasi Silhouette Score
# Visualisasi Silhouette Score dengan diagram batang
plt.figure(figsize=(8, 6))
plt.bar(range(2, 7), silhouette_scores)  # Menggunakan plt.bar untuk diagram batang
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score for Different k Values")
plt.xticks(range(2, 7))  # Menampilkan nilai k pada sumbu x

# Menampilkan nilai score di atas setiap batang
for i, score in enumerate(silhouette_scores):
    plt.text(i + 2, score + 0.005, f"{score:.3f}", ha='center', va='bottom')

plt.grid(True)
plt.show()

best_k = np.argmax(silhouette_scores) + 2  # Menambahkan 2 karena perulangan dimulai dari 2
print(f"The best value of k is : {best_k} with silhouette score {silhouette_scores[best_k-2]}")

In [ ]:
# Perform clustering with the best k value
kmeans = KMeans(n_clusters=best_k, random_state=0)
cluster_labels = kmeans.fit_predict(X)

In [ ]:
def plot_vector_distribution(vector_representation, kmeans_labels):
    # pca = PCA(n_components=2, random_state=0)
    # pca_result = pca.fit_transform(vector_representation)

    # df_pca = pd.DataFrame(pca_result, columns=['Dimension 1', 'Dimension 2'])
    df_pca = pd.DataFrame(vector_representation, columns=['Dimension 1', 'Dimension 2'])
    df_pca['Cluster Label'] = kmeans_labels

    cluster_palette = sns.color_palette('tab10', n_colors=len(np.unique(kmeans_labels)))
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='Dimension 1', y='Dimension 2', hue='Cluster Label', data=df_pca, palette=cluster_palette)

    # Menambahkan centroid ke dalam plot
    centroids = []
    for label in np.unique(kmeans_labels):
        # centroid = np.mean(pca_result[kmeans_labels == label], axis=0)
        centroid = np.mean(vector_representation[kmeans_labels == label], axis=0)
        centroids.append(centroid)
    centroids = np.array(centroids)
    plt.scatter(centroids[:, 0], centroids[:, 1], marker='^', c='red', s=50, label='Centroids')

    plt.title('PCA Latent Representation with Centroids')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.legend()
    plt.savefig('pca_plot.png') # tambahkan baris ini
    plt.show()

plot_vector_distribution(X, cluster_labels)

In [ ]:
import joblib
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(f'/content/drive/MyDrive/skripsi/pca_en/pca_model_{jurnal_id}.pkl'), exist_ok=True)
joblib.dump(pca, f'/content/drive/MyDrive/skripsi/pca_en/pca_model_{jurnal_id}.pkl')

In [ ]:
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(f'/content/drive/MyDrive/skripsi/kmeans_en/kmeans_model_{jurnal_id}.pkl'), exist_ok=True)
joblib.dump(kmeans, f'/content/drive/MyDrive/skripsi/kmeans_en/kmeans_model_{jurnal_id}.pkl')

In [ ]:
# Simpan label cluster
cluster_labels_dir = '/content/drive/MyDrive/skripsi/cluster_labels_en'
os.makedirs(cluster_labels_dir, exist_ok=True)
np.save(os.path.join(cluster_labels_dir, f'cluster_labels_{jurnal_id}.npy'), cluster_labels)

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(X, cluster_labels)
print("Silhouette Score:", silhouette_avg)

In [ ]:
# Save the silhouette score to the specified path
silhouette_scores_dir = '/content/drive/MyDrive/skripsi/silhouette_scores_en'
os.makedirs(silhouette_scores_dir, exist_ok=True)  # Create directory if it doesn't exist
np.save(os.path.join(silhouette_scores_dir, f'silhouette_score_{jurnal_id}.npy'), silhouette_avg)
print(f"Silhouette score saved to: /content/drive/MyDrive/skripsi/silhouette_scores_en/silhouette_score_{jurnal_id}.npy")

In [ ]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

def inject_and_analyze(source_jid, target_jid):
    # Load embeddings dan model untuk jurnal sumber dan target
    source_embeddings_2d = np.load(f'/content/drive/MyDrive/skripsi/embeddings_2d_en/embeddings_2d_{source_jid}.npy')
    target_embeddings_2d = np.load(f'/content/drive/MyDrive/skripsi/embeddings_2d_en/embeddings_2d_{target_jid}.npy')

    source_kmeans = joblib.load(f'/content/drive/MyDrive/skripsi/kmeans_en/kmeans_model_{source_jid}.pkl')
    target_kmeans = joblib.load(f'/content/drive/MyDrive/skripsi/kmeans_en/kmeans_model_{target_jid}.pkl')

    source_pca = joblib.load(f'/content/drive/MyDrive/skripsi/pca_en/pca_model_{source_jid}.pkl')
    target_pca = joblib.load(f'/content/drive/MyDrive/skripsi/pca_en/pca_model_{target_jid}.pkl')

    # Ambil data artikel dari dataframe utama
    source_data = df[df['jid'] == source_jid]

    # Transformasi embeddings source ke ruang target
    projected_embeddings = target_pca.transform(source_pca.inverse_transform(source_embeddings_2d))

    # Prediksi cluster menggunakan model target
    predicted_clusters = target_kmeans.predict(projected_embeddings)

    # Hitung jarak ke centroid masing-masing cluster
    distances = []
    for i, embedding in enumerate(projected_embeddings):
        cluster = predicted_clusters[i]
        centroid = target_kmeans.cluster_centers_[cluster]
        distance = np.linalg.norm(embedding - centroid)
        distances.append(distance)

    # Hitung threshold (mean + 2 * std)
    mean_distance = np.mean(distances)
    std_distance = np.std(distances)
    threshold = mean_distance + 2 * std_distance

    # Klasifikasikan artikel sebagai in-scope atau out-scope
    results = ["In-scope" if d <= threshold else "Out-scope" for d in distances]

    # Buat dataframe hasil analisis
    df_result = pd.DataFrame({
        'Journal ID': source_data['jid'].values,
        'Journal Name': source_data['journal'].values,
        'Article Title': source_data['title'].values,
        'Predicted Cluster': predicted_clusters,
        'Distance to Centroid': distances,
        'Scope': results
    })

    # Visualisasi hasil injeksi
    plt.figure(figsize=(10, 6))
    plt.scatter(target_embeddings_2d[:, 0], target_embeddings_2d[:, 1], alpha=0.5, label='Target Journal Articles', color='blue')
    plt.scatter(projected_embeddings[:, 0], projected_embeddings[:, 1], c=['green' if r == "In-scope" else 'red' for r in results], label='Injected Articles', cmap='coolwarm', edgecolors='black')

    plt.axhline(y=threshold, color='gray', linestyle='--', label='Threshold')
    plt.title(f'Injection Analysis: Journal {source_jid} to {target_jid}')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.legend()
    plt.show()

    return df_result

# Contoh penggunaan:
source_journal_id = 1
target_journal_id = 5

result_df = inject_and_analyze(source_journal_id, target_journal_id)
result_df
